In [ ]:
import numpy as np
from scipy.ndimage import uniform_filter1d
from scipy.signal import detrend


def find_peaks_original(x, scale=None, debug=False):
    """Find peaks in quasi-periodic noisy signals using AMPD algorithm.
    Automatic Multi-Scale Peak Detection originally proposed in
    "An Efficient Algorithm for Automatic Peak Detection in
    Noisy Periodic and Quasi-Periodic Signals", Algorithms 2012, 5, 588-603
    https://doi.org/10.1109/ICRERA.2016.7884365
    Optimized implementation by Igor Gotlibovych, 2018
    Parameters
    ----------
    x : ndarray
        1-D array on which to find peaks
    scale : int, optional
        specify maximum scale window size of (2 * scale + 1)
    debug : bool, optional
        if set to True, return the Local Scalogram Matrix, `LSM`,
        and scale with most local maxima, `l`,
        together with peak locations
    Returns
    -------
    pks: ndarray
        The ordered array of peak indices found in `x`
    """
    x = detrend(x)
    N = len(x)
    L = N // 2
    if scale:
        L = min(scale, L)

    # create LSM matix
    LSM = np.zeros((L, N), dtype=bool)
    for k in np.arange(1, L):
        LSM[k - 1, k:N - k] = (
            (x[0:N - 2 * k] < x[k:N - k]) & (x[k:N - k] > x[2 * k:N])
        )

    # Find scale with most maxima
    G = LSM.sum(axis=1)
    l_scale = np.argmax(G)

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM[0:l_scale, :], axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, LSM, l_scale
    return pks


def find_peaks(x, scale=None, debug=False):
    """Find peaks in quasi-periodic noisy signals using AMPD algorithm.
    Extended implementation handles peaks near start/end of the signal.
    Optimized implementation by Igor Gotlibovych, 2018
    Parameters
    ----------
    x : ndarray
        1-D array on which to find peaks
    scale : int, optional
        specify maximum scale window size of (2 * scale + 1)
    debug : bool, optional
        if set to True, return the Local Scalogram Matrix, `LSM`,
        weigted number of maxima, 'G',
        and scale at which G is maximized, `l`,
        together with peak locations
    Returns
    -------
    pks: ndarray
        The ordered array of peak indices found in `x`
    """
    x = detrend(x)
    N = len(x)
    L = N // 2
    if scale:
        L = min(scale, L)

    # create LSM matix
    LSM = np.ones((L, N), dtype=bool)
    for k in np.arange(1, L + 1):
        LSM[k - 1, 0:N - k] &= (x[0:N - k] > x[k:N]
                                )  # compare to right neighbours
        LSM[k - 1, k:N] &= (x[k:N] > x[0:N - k])  # compare to left neighbours

    # Find scale with most maxima
    G = LSM.sum(axis=1)
    G = G * np.arange(
        N // 2, N // 2 - L, -1
    )  # normalize to adjust for new edge regions
    l_scale = np.argmax(G)

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM[0:l_scale, :], axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, LSM, G, l_scale
    return pks


def find_peaks_adaptive(x, window=None, debug=False):
    """Find peaks in quasi-periodic noisy signals using ASS-AMPD algorithm.
    Adaptive Scale Selection Automatic Multi-Scale Peak Detection,
    an extension of AMPD -
    "An Efficient Algorithm for Automatic Peak Detection in
    Noisy Periodic and Quasi-Periodic Signals", Algorithms 2012, 5, 588-603
    https://doi.org/10.1109/ICRERA.2016.7884365
    Optimized implementation by Igor Gotlibovych, 2018
    Parameters
    ----------
    x : ndarray
        1-D array on which to find peaks
    window : int, optional
        sliding window size for adaptive scale selection
    debug : bool, optional
        if set to True, return the Local Scalogram Matrix, `LSM`,
        and `adaptive_scale`,
        together with peak locations
    Returns
    -------
    pks: ndarray
        The ordered array of peak indices found in `x`
    """
    x = detrend(x)
    N = len(x)
    if not window:
        window = N
    if window > N:
        window = N
    L = window // 2

    # create LSM matix
    LSM = np.ones((L, N), dtype=bool)
    for k in np.arange(1, L + 1):
        LSM[k - 1, 0:N - k] &= (x[0:N - k] > x[k:N]
                                )  # compare to right neighbours
        LSM[k - 1, k:N] &= (x[k:N] > x[0:N - k])  # compare to left neighbours

    # Create continuos adaptive LSM
    ass_LSM = uniform_filter1d(LSM * window, window, axis=1, mode='nearest')
    normalization = np.arange(L, 0, -1)  # scale normalization weight
    ass_LSM = ass_LSM * normalization.reshape(-1, 1)

    # Find adaptive scale at each point
    adaptive_scale = ass_LSM.argmax(axis=0)

    # construct reduced LSM
    LSM_reduced = LSM[:adaptive_scale.max(), :]
    mask = (np.indices(LSM_reduced.shape)[0] > adaptive_scale
            )  # these elements are outside scale of interest
    LSM_reduced[mask] = 1

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM_reduced, axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, ass_LSM, adaptive_scale
    return pks

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.utils.data as Data
import torchvision.datasets
import torchvision.transforms
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import glob
# from pyampd.ampd import find_peaks


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

import matplotlib.pyplot as plt


import joblib

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/archive.zip

Archive:  gdrive/My Drive/archive.zip
  inflating: Samples/rec_1.csv       
  inflating: Samples/rec_10.csv      
  inflating: Samples/rec_100.csv     
  inflating: Samples/rec_101.csv     
  inflating: Samples/rec_102.csv     
  inflating: Samples/rec_103.csv     
  inflating: Samples/rec_104.csv     
  inflating: Samples/rec_105.csv     
  inflating: Samples/rec_106.csv     
  inflating: Samples/rec_107.csv     
  inflating: Samples/rec_108.csv     
  inflating: Samples/rec_109.csv     
  inflating: Samples/rec_11.csv      
  inflating: Samples/rec_110.csv     
  inflating: Samples/rec_111.csv     
  inflating: Samples/rec_112.csv     
  inflating: Samples/rec_113.csv     
  inflating: Samples/rec_114.csv     
  inflating: Samples/rec_115.csv     
  inflating: Samples/rec_116.csv     
  inflating: Samples/rec_117.csv     
  inflating: Samples/rec_118.csv     
  inflating: Samples/rec_119.csv     
  inflating: Samples/rec_12.csv      
  inflating: Samples/rec_120.csv     
  inflating:

In [ ]:
PPG_datas = []
ABP_datas = []
ECG_datas = []
i = 0
for name in glob.glob('./Samples/*.csv'):
    raw_training_data = pd.read_csv(name, header=None)
    raw_training_data = np.array(raw_training_data)
    #print(raw_training_data.shape)
    PPG_data = raw_training_data[0].reshape(-1)
    ABP_data = raw_training_data[1].reshape(-1)
    ECG_data = raw_training_data[2].reshape(-1)

    PPG_datas.append(PPG_data)
    ABP_datas.append(ABP_data)
    ECG_datas.append(ECG_data)
    # i = i + 1
 #   print(i)
#     if (i == 100):
#         break

PPG_datas = np.array(PPG_datas)
ABP_datas = np.array(ABP_datas)
ECG_datas = np.array(ECG_datas)
# raw_training_data.head()

<ipython-input-5-11da2678ca3e>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  PPG_datas = np.array(PPG_datas)
<ipython-input-5-11da2678ca3e>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ABP_datas = np.array(ABP_datas)
<ipython-input-5-11da2678ca3e>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ECG_datas = np.array(ECG_da

In [ ]:
def generate_whole_based_vector(X):
    vector = np.zeros(4 * SAMPLE_FREQ)
    vector[:len(X)] = X
    return vector

In [ ]:
SAMPLE_FREQ = 125

In [ ]:
whole_based_vectors = []
SBP_data = []
DBP_data = []
MAP_data = []

for j in range(len(PPG_datas)):
    sec_15 = 15*SAMPLE_FREQ
    PPG_data = PPG_datas[j]
    ABP_data = ABP_datas[j]
    PPG_peaks = find_peaks(PPG_data, scale=SAMPLE_FREQ)
    for i in range(2, PPG_peaks.shape[0]):
        X = PPG_data[PPG_peaks[i-1]:PPG_peaks[i]]
        if(len(X) < SAMPLE_FREQ):
            whole_based_vector = generate_whole_based_vector(X)

            SBP = np.max(ABP_data[PPG_peaks[i-1]:PPG_peaks[i-1]+sec_15])
            DBP = np.min(ABP_data[PPG_peaks[i-1]:PPG_peaks[i-1]+sec_15])
            MAP = SBP/3 + 2*DBP/3

            whole_based_vectors.append(whole_based_vector)
            SBP_data.append(SBP)
            DBP_data.append(DBP)
            MAP_data.append(MAP)

In [ ]:
whole_based_vectors = np.array(whole_based_vectors)
SBP_data = np.array(SBP_data)
DBP_data = np.array(DBP_data)
MAP_data = np.array(MAP_data)
print(whole_based_vectors.shape)
print(SBP_data.shape)

(196625, 500)
(196625,)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=43)
pca_whole_based_vectors = pca.fit_transform(whole_based_vectors)

print(pca_whole_based_vectors.shape)

(196625, 43)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, SBP_train, SBP_test, DBP_train, DBP_test, MAP_train, MAP_test = train_test_split(
                                            pca_whole_based_vectors, SBP_data, DBP_data, MAP_data, test_size=0.1, random_state=42)

In [ ]:
def AAMI_standard(predict, test):
    total = len(predict)
    ME = np.mean(predict - test)
    MAE = np.mean(np.abs(predict-test))
    SD = np.std(predict-test)

    return total, ME, MAE, SD

def BHS_standard(predict, test):
    total = len(predict)
    mm5 = np.sum(np.abs(predict-test)<=5)
    mm10 = np.sum(np.abs(predict-test)<=10)
    mm15 = np.sum(np.abs(predict-test)<=15)
    return total, mm5, mm10, mm15

In [ ]:
EPOCHS = 2
LR = 0.001

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTM_Attention(nn.Module):
    def __init__(self, input_size, hidden_dim=256, num_layers=1, dropout=0.2):
        super(LSTM_Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = nn.Dropout(p=dropout)
        self.activation = nn.ReLU()
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1, bias=False),
            nn.Softmax(dim=1)
        )
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        self.hidden1 = nn.Linear(in_features=hidden_dim, out_features=512)
        self.hidden2 = nn.Linear(in_features=512, out_features=256)
        self.hidden3 = nn.Linear(in_features=256, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attn_weights = self.attention(lstm_out)
        context = attn_weights * lstm_out
        context = torch.sum(context, dim=1)
        out = self.hidden1(self.activation(context))
        out = self.dropout(out)
        out = self.hidden2(self.activation(out))
        out = self.hidden3(self.activation(out))
        out = self.output(self.activation(out))
        return out

    def get_weight(self):
        weight = []
        weight.append(self.lstm.weight_ih_l0)
        weight.append(self.lstm.weight_hh_l0)
        weight.append(self.lstm.bias_ih_l0)
        weight.append(self.lstm.bias_hh_l0)
        weight.append(self.hidden1.weight)
        weight.append(self.hidden2.weight)
        weight.append(self.hidden3.weight)
        weight.append(self.output.weight)
        return weight

    def attention_weights(self, x):
        lstm_out, _ = self.lstm(x)
        attn_weights = self.attention(lstm_out)
        return attn_weights


In [ ]:
class Module:
    def __init__(self, net, train_loader, test_loader, EPOCH=20, LR=0.0001):
        self.net = net
        self.optimizer = torch.optim.Adam(net.parameters(), lr=LR)
        self.criterion = nn.MSELoss()
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.EPOCHS = EPOCH
        self.LR = LR

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.net = self.net.to(self.device)

    def start(self):
        self.history_train_loss = []
        self.history_test_loss = []
        for epoch in range(self.EPOCHS):
            print('Epoch:', epoch)
            train_loss = self.train()
            test_loss = self.test()

            self.history_train_loss.append(train_loss)
            self.history_test_loss.append(test_loss)

    def train(self):
        self.net.train()
        train_loss = 0
        for step, (batch_X, batch_y) in enumerate(self.train_loader):
            batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.net(batch_X).double()

            # Get the attention weights
            alpha = self.net.attention_weights(batch_X).squeeze(1)

            # Calculate the predicted values using attention weights
            pred = torch.sum(alpha * outputs, dim=1)

            loss = self.criterion(pred, batch_y)
            train_loss += loss.item()
            loss.backward()
            self.optimizer.step()
        train_loss /= len(self.train_loader.dataset)
        print('【Training】Loss: %.3f' % train_loss)
        return train_loss

    def test(self):
        self.net.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for step, (batch_X, batch_y) in enumerate(self.test_loader):
                batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
                outputs = self.net(batch_X).double()

                # Get the attention weights
                alpha = self.net.attention_weights(batch_X).squeeze(1)

                # Calculate the predicted values using attention weights
                pred = torch.sum(alpha * outputs, dim=1)

                loss = self.criterion(pred, batch_y)
                test_loss += loss.item()

                total += batch_y.size(0)
                correct += torch.sum((pred > 0.5) == (batch_y > 0.5)).item()

        test_loss /= len(self.test_loader.dataset)
        accuracy = 100 * correct / total

        print('【Testing】Loss: {:.4f} | Accuracy: {:.2f}%'.format(test_loss, accuracy))
        return test_loss, accuracy

    def predict(self, test_loader):
        outputs = []
        self.net.eval()

        with torch.no_grad():
            for step, (batch_X, batch_y) in enumerate(test_loader):
                print('batch_X shape:', batch_X.shape)
                print('batch_y shape:', batch_y.shape)

                batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)

                # Get the attention weights
                alpha = self.net.attention_weights(batch_X).squeeze(1)
                print('alpha shape:', alpha.shape)

                # Calculate the predicted values using attention weights
                output = self.net(batch_X).double().cpu().numpy().reshape(-1)
                print('output shape:', output.shape)
                outputs.extend(output)

        return outputs

    def get_weight(self):
        return self.net.get_weight()

    def get_model(self):
        return self.net.state_dict()



In [ ]:
from sklearn.model_selection import train_test_split

L = 43

x_train, x_test, SBP_train, SBP_test, DBP_train, DBP_test, MAP_train, MAP_test = train_test_split(
                                            pca_whole_based_vectors, SBP_data, DBP_data, MAP_data, test_size=0.1, random_state=42)


x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
SBP_train = torch.from_numpy(SBP_train)
SBP_test = torch.from_numpy(SBP_test)
DBP_train = torch.from_numpy(DBP_train)
DBP_test = torch.from_numpy(DBP_test)
MAP_train = torch.from_numpy(MAP_train)
MAP_test = torch.from_numpy(MAP_test)

x_train = x_train.view(-1, 1, L)
x_test = x_test.view(-1, 1, L)

print(x_train.shape, x_test.shape, SBP_train.shape, SBP_test.shape)

SBP_train_dataset = Data.TensorDataset(x_train, SBP_train)
SBP_test_dataset = Data.TensorDataset(x_test, SBP_test)


SBP_train_loader = Data.DataLoader(
    dataset = SBP_train_dataset,
    batch_size = 128,
)

SBP_test_loader = Data.DataLoader(
    dataset = SBP_test_dataset,
    batch_size = 10,
)

DBP_train_dataset = Data.TensorDataset(x_train, DBP_train)
DBP_test_dataset = Data.TensorDataset(x_test, DBP_test)


DBP_train_loader = Data.DataLoader(
    dataset = DBP_train_dataset,
    batch_size = 128,
)

DBP_test_loader = Data.DataLoader(
    dataset = DBP_test_dataset,
    batch_size = 10,
)

MAP_train_dataset = Data.TensorDataset(x_train, MAP_train)
MAP_test_dataset = Data.TensorDataset(x_test, MAP_test)


MAP_train_loader = Data.DataLoader(
    dataset = MAP_train_dataset,
    batch_size = 128,
)

MAP_test_loader = Data.DataLoader(
    dataset = MAP_test_dataset,
    batch_size = 10,
)

torch.Size([176962, 1, 43]) torch.Size([19663, 1, 43]) torch.Size([176962]) torch.Size([19663])


In [ ]:
SBP_lstm_module = Module(LSTM_Attention(input_size=L), SBP_train_loader, SBP_test_loader, EPOCH=200, LR=LR)
SBP_lstm_module.start()

Epoch: 0
【Training】Loss: 6.743
【Testing】Loss: 30.6305 | Accuracy: 100.00%
Epoch: 1
【Training】Loss: 2.346
【Testing】Loss: 27.0574 | Accuracy: 100.00%
Epoch: 2
【Training】Loss: 2.148
【Testing】Loss: 25.5598 | Accuracy: 100.00%
Epoch: 3
【Training】Loss: 2.052
【Testing】Loss: 24.8264 | Accuracy: 100.00%
Epoch: 4
【Training】Loss: 1.974
【Testing】Loss: 24.4913 | Accuracy: 100.00%
Epoch: 5
【Training】Loss: 1.923
【Testing】Loss: 23.6962 | Accuracy: 100.00%
Epoch: 6
【Training】Loss: 1.873
【Testing】Loss: 23.3314 | Accuracy: 100.00%
Epoch: 7
【Training】Loss: 1.831
【Testing】Loss: 23.1270 | Accuracy: 100.00%
Epoch: 8
【Training】Loss: 1.791
【Testing】Loss: 22.5792 | Accuracy: 100.00%
Epoch: 9
【Training】Loss: 1.764
【Testing】Loss: 22.0748 | Accuracy: 100.00%
Epoch: 10
【Training】Loss: 1.735
【Testing】Loss: 21.9343 | Accuracy: 100.00%
Epoch: 11
【Training】Loss: 1.715
【Testing】Loss: 21.8150 | Accuracy: 100.00%
Epoch: 12
【Training】Loss: 1.688
【Testing】Loss: 21.3917 | Accuracy: 100.00%
Epoch: 13
【Training】Loss: 1.665
【Te

In [ ]:
DBP_lstm_module = Module(LSTM_Attention(input_size=L), DBP_train_loader, DBP_test_loader, EPOCH=200, LR=LR)
DBP_lstm_module.start()

Epoch: 0
【Training】Loss: 1.257
【Testing】Loss: 8.4502 | Accuracy: 100.00%
Epoch: 1
【Training】Loss: 0.616
【Testing】Loss: 8.9649 | Accuracy: 100.00%
Epoch: 2
【Training】Loss: 0.585
【Testing】Loss: 8.6782 | Accuracy: 100.00%
Epoch: 3
【Training】Loss: 0.563
【Testing】Loss: 8.4039 | Accuracy: 100.00%
Epoch: 4
【Training】Loss: 0.544
【Testing】Loss: 7.9418 | Accuracy: 100.00%
Epoch: 5
【Training】Loss: 0.529
【Testing】Loss: 7.7053 | Accuracy: 100.00%
Epoch: 6
【Training】Loss: 0.517
【Testing】Loss: 7.2926 | Accuracy: 100.00%
Epoch: 7
【Training】Loss: 0.504
【Testing】Loss: 7.0485 | Accuracy: 100.00%
Epoch: 8
【Training】Loss: 0.493
【Testing】Loss: 6.9824 | Accuracy: 100.00%
Epoch: 9
【Training】Loss: 0.483
【Testing】Loss: 6.5950 | Accuracy: 100.00%
Epoch: 10
【Training】Loss: 0.473
【Testing】Loss: 6.4234 | Accuracy: 100.00%
Epoch: 11
【Training】Loss: 0.465
【Testing】Loss: 6.2222 | Accuracy: 100.00%
Epoch: 12
【Training】Loss: 0.456
【Testing】Loss: 6.2579 | Accuracy: 100.00%
Epoch: 13
【Training】Loss: 0.449
【Testing】Loss: 5

In [ ]:
MAP_lstm_module = Module(LSTM_Attention(input_size=L), MAP_train_loader, MAP_test_loader, EPOCH=200, LR=LR)
MAP_lstm_module.start()

Epoch: 0
【Training】Loss: 2.177
【Testing】Loss: 9.9747 | Accuracy: 100.00%
Epoch: 1
【Training】Loss: 0.748
【Testing】Loss: 9.1317 | Accuracy: 100.00%
Epoch: 2
【Training】Loss: 0.692
【Testing】Loss: 8.5211 | Accuracy: 100.00%
Epoch: 3
【Training】Loss: 0.656
【Testing】Loss: 8.5358 | Accuracy: 100.00%
Epoch: 4
【Training】Loss: 0.630
【Testing】Loss: 8.0026 | Accuracy: 100.00%
Epoch: 5
【Training】Loss: 0.612
【Testing】Loss: 8.0207 | Accuracy: 100.00%
Epoch: 6
【Training】Loss: 0.595
【Testing】Loss: 8.1297 | Accuracy: 100.00%
Epoch: 7
【Training】Loss: 0.581
【Testing】Loss: 7.5180 | Accuracy: 100.00%
Epoch: 8
【Training】Loss: 0.567
【Testing】Loss: 7.1566 | Accuracy: 100.00%
Epoch: 9
【Training】Loss: 0.556
【Testing】Loss: 7.2376 | Accuracy: 100.00%
Epoch: 10
【Training】Loss: 0.546
【Testing】Loss: 7.2682 | Accuracy: 100.00%
Epoch: 11
【Training】Loss: 0.535
【Testing】Loss: 7.3615 | Accuracy: 100.00%
Epoch: 12
【Training】Loss: 0.525
【Testing】Loss: 7.0314 | Accuracy: 100.00%
Epoch: 13
【Training】Loss: 0.515
【Testing】Loss: 7

In [ ]:
# temp_predict = []
# for predicts in SBP_predict:
#     for i in range(len(predicts))
#         temp_predict.append(predicts[i])
# print(SBP_predict[0:20])

# print(SBP_test[0:20])

SBP_predict = np.array(SBP_lstm_module.predict(SBP_test_loader)).reshape(-1)
SBP_test = np.array(SBP_test)
DBP_predict = np.array(DBP_lstm_module.predict(DBP_test_loader)).reshape(-1)
DBP_test = np.array(DBP_test)
MAP_predict = np.array(MAP_lstm_module.predict(MAP_test_loader)).reshape(-1)
MAP_test = np.array(MAP_test)

print('LSTM_Attention')

total, ME, MAE, SD = AAMI_standard(SBP_predict, SBP_test)
print()
print()
print('----------------SBP-----------------')
print()
print('-------------AAMI standard----------')
print('ME             MAE             SD           total')
print('%.3f         %.3f         %.3f          %d' % (ME,MAE,SD,total))
#print('ME: ', ME, '   MAE: ', MAE, '   SD: ', SD, '   total: ', total)
total, mm5, mm10, mm15 = BHS_standard(SBP_predict, SBP_test)
print()
print('-------------BHS standard------------')
print('<5mmHg        <10mmHg        <15mmHg        total')
print('%d            %d           %d          %d' % (mm5,mm10,mm15,total))
print('%.3f%s        %.3f%s       %.3f%s         %d' % (mm5/total*100, '%',mm10/total*100, '%',mm15/total*100, '%', total))

total, ME, MAE, SD = AAMI_standard(DBP_predict, DBP_test)
print()
print()
print('----------------DBP-----------------')
print()
print('-------------AAMI standard----------')
print('ME             MAE             SD           total')
print('%.3f         %.3f         %.3f          %d' % (ME,MAE,SD,total))
#print('ME: ', ME, '   MAE: ', MAE, '   SD: ', SD, '   total: ', total)
total, mm5, mm10, mm15 = BHS_standard(DBP_predict, DBP_test)
print()
print('-------------BHS standard------------')
print('<5mmHg        <10mmHg        <15mmHg        total')
print('%d            %d           %d          %d' % (mm5,mm10,mm15,total))
print('%.3f%s        %.3f%s       %.3f%s         %d' % (mm5/total*100, '%',mm10/total*100, '%',mm15/total*100, '%', total))

total, ME, MAE, SD = AAMI_standard(MAP_predict, MAP_test)
print()
print()
print('----------------MAP-----------------')
print()
print('-------------AAMI standard----------')
print('ME             MAE             SD           total')
print('%.3f         %.3f         %.3f          %d' % (ME,MAE,SD,total))
#print('ME: ', ME, '   MAE: ', MAE, '   SD: ', SD, '   total: ', total)
total, mm5, mm10, mm15 = BHS_standard(MAP_predict, MAP_test)
print()
print('-------------BHS standard------------')
print('<5mmHg        <10mmHg        <15mmHg        total')
print('%d            %d           %d          %d' % (mm5,mm10,mm15,total))
print('%.3f%s        %.3f%s       %.3f%s         %d' % (mm5/total*100, '%',mm10/total*100, '%',mm15/total*100, '%', total))

Streaming output truncated to the last 5000 lines.
batch_y shape: torch.Size([10])
alpha shape: torch.Size([10, 1])
output shape: (10,)
batch_X shape: torch.Size([10, 1, 43])
batch_y shape: torch.Size([10])
alpha shape: torch.Size([10, 1])
output shape: (10,)
batch_X shape: torch.Size([10, 1, 43])
batch_y shape: torch.Size([10])
alpha shape: torch.Size([10, 1])
output shape: (10,)
batch_X shape: torch.Size([10, 1, 43])
batch_y shape: torch.Size([10])
alpha shape: torch.Size([10, 1])
output shape: (10,)
batch_X shape: torch.Size([10, 1, 43])
batch_y shape: torch.Size([10])
alpha shape: torch.Size([10, 1])
output shape: (10,)
batch_X shape: torch.Size([10, 1, 43])
batch_y shape: torch.Size([10])
alpha shape: torch.Size([10, 1])
output shape: (10,)
batch_X shape: torch.Size([10, 1, 43])
batch_y shape: torch.Size([10])
alpha shape: torch.Size([10, 1])
output shape: (10,)
batch_X shape: torch.Size([10, 1, 43])
batch_y shape: torch.Size([10])
alpha shape: torch.Size([10, 1])
output shape: (1

In [ ]:
print('LSTM_Attention')

SBP_cc= np.corrcoef(SBP_predict, SBP_test)
DBP_cc= np.corrcoef(DBP_predict, DBP_test)
MAP_cc= np.corrcoef(MAP_predict, MAP_test)
print()
print()
print('------------Correlation Coefficient-------------')
print()
print('SBP: %.3f' % (SBP_cc[0, 1]))
print()
print('DBP: %.3f' % (DBP_cc[0, 1]))
print()
print('MAP: %.3f' % (MAP_cc[0, 1]))

LSTM_Attention


------------Correlation Coefficient-------------

SBP: 0.817

DBP: 0.780

MAP: 0.816
